### Lets install all the dependencies first

First we are going to install **Hugging Face**

In [1]:
pip install transformers

     |████████████████████████████████| 1.4MB 8.2MB/s 
     |████████████████████████████████| 890kB 49.8MB/s 
     |████████████████████████████████| 2.9MB 50.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=936376b3db4fb6a391e77e8b1199adaa2e457734ba522833132413ed30cafcba
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
pip install -q ktrain

     |████████████████████████████████| 25.3MB 137kB/s 
     |████████████████████████████████| 983kB 52.1MB/s 
     |████████████████████████████████| 266kB 50.0MB/s 
     |████████████████████████████████| 1.3MB 51.6MB/s 
     |████████████████████████████████| 1.1MB 49.9MB/s 
     |████████████████████████████████| 471kB 49.9MB/s 
     |████████████████████████████████| 2.9MB 52.8MB/s 
ERROR: transformers 3.5.1 has requirement sentencepiece==0.1.91, but you'll have sentencepiece 0.1.94 which is incompatible.


We now install dependencies for **NLTK**

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
!git clone https://github.com/wangcunxiang/SemEval2020-Task4-Commonsense-Validation-and-Explanation.git

Cloning into 'SemEval2020-Task4-Commonsense-Validation-and-Explanation'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 88 (delta 36), reused 64 (delta 19), pack-reused 0
Unpacking objects: 100% (88/88), done.


### Now we will import libraries and Prepare the models

In [5]:
from nltk import pos_tag, word_tokenize
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import keras
import math
import torch
import pickle
from keras.models import Model
import keras.backend as K
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
import itertools
from keras.models import load_model
from sklearn.utils import shuffle
from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig
from transformers import OpenAIGPTTokenizer, OpenAIGPTLMHeadModel
import ktrain
from ktrain import text
print('Tensorflow version: ',tf.__version__)

Tensorflow version:  2.3.0


**GPT** Model

In [6]:
model_gpt = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt')
model_gpt.eval()
tokenizer_gpt = OpenAIGPTTokenizer.from_pretrained('openai-gpt')

**BERT** Model

In [7]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")
bert_model = TFBertForSequenceClassification.from_pretrained('bert-large-uncased',num_labels=2)

**DistilBert** Model

In [8]:
MODEL_NAME = 'distilbert-base-uncased'

### Lets load the data 

Train Data

In [9]:
df = pd.read_csv ('SemEval2020-Task4-Commonsense-Validation-and-Explanation/''ALL data''/''Training  Data''/subtaskA_data_all.csv', header = None)
traindata =  df.to_numpy()
df = pd.read_csv ('SemEval2020-Task4-Commonsense-Validation-and-Explanation/''ALL data''/''Training  Data''/subtaskA_answers_all.csv', header = None)
trainlabels =  df.to_numpy()

Validation Data

In [10]:
df = pd.read_csv ('SemEval2020-Task4-Commonsense-Validation-and-Explanation/''ALL data''/''Dev Data''/subtaskA_dev_data.csv', header = None)
valdata =  df.to_numpy()
df = pd.read_csv ('SemEval2020-Task4-Commonsense-Validation-and-Explanation/''ALL data''/''Dev Data''/subtaskA_gold_answers.csv', header = None)
vallabels =  df.to_numpy()

Test Data

In [11]:
df = pd.read_csv ('SemEval2020-Task4-Commonsense-Validation-and-Explanation/''ALL data''/''Test Data''/subtaskA_test_data.csv', header = None)
testdata =  df.to_numpy()
df = pd.read_csv ('SemEval2020-Task4-Commonsense-Validation-and-Explanation/''ALL data''/''Test Data''/subtaskA_gold_answers.csv', header = None)
testlabels =  df.to_numpy()

Getting train data ready

In [12]:
labels = []
for i in range(len(trainlabels)):
  if trainlabels[i][1] == 0:
    labels.append(0)
    labels.append(1)
  else:
    labels.append(1)
    labels.append(0)
sentenceid = [i[0] for i in traindata[1:]]
sentence0 = [i[1] for i in traindata[1:]]
sentence1 = [i[2] for i in traindata[1:]]
sentences = []
for i in range(len(sentence0)):
  sentences.append(sentence0[i])
  sentences.append(sentence1[i])
# print('ID\tSentence\tLabels')
sentid=[]
for i in range(len(sentenceid)):
  sentid.append(sentenceid[i])
  sentid.append(sentenceid[i])
# for i in range(20):
#   print(sentid[i],sentences[i],labels[i])

Getting validation data ready

In [13]:
devlabels = []
for i in range(len(vallabels)):
  if vallabels[i][1] == 0:
    devlabels.append(0)
    devlabels.append(1)
  else:
    devlabels.append(1)
    devlabels.append(0)

devsentenceid = [i[0] for i in valdata[1:]]
devsentence0 = [i[1] for i in valdata[1:]]
devsentence1 = [i[2] for i in valdata[1:]]
devsentences = []
for i in range(len(devsentence0)):
  devsentences.append(devsentence0[i])
  devsentences.append(devsentence1[i])

# print('ID\tSentence\tLabels')
devsentid=[]
for i in range(len(devsentenceid)):
  devsentid.append(devsentenceid[i])
  devsentid.append(devsentenceid[i])
# for i in range(20):
#   print(devsentid[i],devsentences[i],devlabels[i])

Getting test data ready

In [14]:
testlabel = []
for i in range(len(testlabels)):
  if testlabels[i][1] == 0:
    testlabel.append(0)
    testlabel.append(1)
  else:
    testlabel.append(1)
    testlabel.append(0)

testsentenceid = [i[0] for i in testdata[1:]]
testsentence0 = [i[1] for i in testdata[1:]]
testsentence1 = [i[2] for i in testdata[1:]]
testsentences = []
for i in range(len(testsentence0)):
  testsentences.append(testsentence0[i])
  testsentences.append(testsentence1[i])

# print('ID\tSentence\tLabels')
testsentid=[]
for i in range(len(testsentenceid)):
  testsentid.append(testsentenceid[i])
  testsentid.append(testsentenceid[i])
# for i in range(20):
#   print(testsentid[i],testsentences[i],testlabels[i])

###**GPT**
As GPT is Pretrained we don't need to train out data we can get the preplexity and compare the values of both sentences and the one with higher score will be against common sense

In [15]:
def perp_score(sentence):
    tokenize_input = tokenizer_gpt.tokenize(sentence)
    tensor_input = torch.tensor([tokenizer_gpt.convert_tokens_to_ids(tokenize_input)])
    loss = model_gpt(tensor_input, labels=tensor_input)
    return math.exp(loss[0])
gpt_pred = []
for i in range(len(testsentenceid)):
  sent = [testsentence0[i],testsentence1[i]]
  score = [perp_score(i) for i in sent]
  if score[1]>score[0]:
    gpt_pred.append(1)
  else:
    gpt_pred.append(0)

GPT Accuracy

In [16]:
gpt_correct = 0
for i in range(len(gpt_pred)):
  if gpt_pred[i] == testlabels[i][1]: gpt_correct +=1
print('GPT Accuracy:',(gpt_correct/len(testsentenceid))*100,'%')

GPT Accuracy: 75.0 %


###**BERT**

Convert Sentences into Input IDs

In [17]:
train_input_ids=[]
train_attention_masks=[]

for sent in sentences:
    bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =64,pad_to_max_length = True,return_attention_mask = True)
    train_input_ids.append(bert_inp['input_ids'])
    train_attention_masks.append(bert_inp['attention_mask'])

train_input_ids=np.asarray(train_input_ids)
train_attention_masks=np.array(train_attention_masks)
train_labels=np.array(labels)

In [18]:
val_input_ids=[]
val_attention_masks=[]

for sent in devsentences:
    bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =64,pad_to_max_length = True,return_attention_mask = True)
    val_input_ids.append(bert_inp['input_ids'])
    val_attention_masks.append(bert_inp['attention_mask'])

val_input_ids=np.asarray(val_input_ids)
val_attention_masks=np.array(val_attention_masks)
val_labels=np.array(devlabels)

In [19]:
testinput_ids=[]
testattention_masks=[]

for sent in testsentences:
    bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =32,pad_to_max_length = True,return_attention_mask = True)
    testinput_ids.append(bert_inp['input_ids'])
    testattention_masks.append(bert_inp['attention_mask'])

testinput_ids=np.asarray(testinput_ids)
testattention_masks=np.array(testattention_masks)
testabels=np.array(testlabels)

In [20]:
train_inp = train_input_ids
val_inp = val_input_ids
train_label = train_labels
val_label = val_labels
train_mask = train_attention_masks
val_mask = val_attention_masks


print('Train inp shape {} Val input shape {}\nTrain label shape {} Val label shape {}\nTrain attention mask shape {} Val attention mask shape {}'.format(train_inp.shape,val_inp.shape,train_label.shape,val_label.shape,train_mask.shape,val_mask.shape))

Train inp shape (20000, 64) Val input shape (1994, 64)
Train label shape (20000,) Val label shape (1994,)
Train attention mask shape (20000, 64) Val attention mask shape (1994, 64)


Set Parameteres

In [21]:
print('\nBert Model',bert_model.summary())

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

bert_model.compile(loss=loss,optimizer=optimizer,metrics=[metric])

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  335141888 
_________________________________________________________________
dropout_73 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  2050      
Total params: 335,143,938
Trainable params: 335,143,938
Non-trainable params: 0
_________________________________________________________________

Bert Model None


Training

In [22]:
history=bert_model.fit([train_inp,train_mask],train_label,batch_size=32,epochs=8,validation_data=([val_inp,val_mask],val_label))

Epoch 1/8
625/625 [==============================] - 794s 1s/step - loss: 0.5664 - accuracy: 0.6926 - val_loss: 0.4426 - val_accuracy: 0.7818
Epoch 2/8
625/625 [==============================] - 789s 1s/step - loss: 0.3906 - accuracy: 0.8180 - val_loss: 0.3744 - val_accuracy: 0.8265
Epoch 3/8
625/625 [==============================] - 788s 1s/step - loss: 0.2642 - accuracy: 0.8852 - val_loss: 0.4488 - val_accuracy: 0.8159
Epoch 4/8
625/625 [==============================] - 788s 1s/step - loss: 0.1741 - accuracy: 0.9276 - val_loss: 0.4675 - val_accuracy: 0.8375
Epoch 5/8
625/625 [==============================] - 789s 1s/step - loss: 0.1280 - accuracy: 0.9498 - val_loss: 0.5170 - val_accuracy: 0.8345
Epoch 6/8
625/625 [==============================] - 788s 1s/step - loss: 0.0873 - accuracy: 0.9682 - val_loss: 0.5883 - val_accuracy: 0.8385
Epoch 7/8
625/625 [==============================] - 788s 1s/step - loss: 0.0753 - accuracy: 0.9722 - val_loss: 0.6909 - val_accuracy: 0.8310
Epoch 

Get predictions

In [23]:
bert_preds = bert_model.predict([testinput_ids,testattention_masks])
bert_pred_labels = bert_preds[0].argmax(axis=1)
bert_pred = []
i=0
while i<len(bert_pred_labels):
  if bert_pred_labels[i] == 0:
    bert_pred.append(0)
    i += 2
  else:
    bert_pred.append(1)
    i += 2
print('Classification Report')
print(classification_report(testlabel,bert_pred_labels))

Classification Report
              precision    recall  f1-score   support

           0       0.83      0.80      0.82      1000
           1       0.81      0.84      0.82      1000

    accuracy                           0.82      2000
   macro avg       0.82      0.82      0.82      2000
weighted avg       0.82      0.82      0.82      2000



###*DistilBert**

In [24]:
db_train_labels = np.array(labels)
db_val_labels = np.array(devlabels)

In [25]:
print('size of training set: %s' % (len(sentences)))
print('size of validation set: %s' % (len(devsentences)))
print('classes: %s' % (np.unique(labels)))

x_train = sentences
y_train = db_train_labels
x_test = devsentences
y_test = db_val_labels

size of training set: 20000
size of validation set: 1994
classes: [0 1]


Prepare Model and parameter

In [26]:
t = text.Transformer(MODEL_NAME, maxlen=500, classes=np.unique(labels))
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:414: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')



preprocessing train...
language: en
train sequence lengths:
	mean : 8
	95percentile : 13
	99percentile : 16


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 7
	95percentile : 12
	99percentile : 15


Training

In [27]:
learner.fit_onecycle(5e-5, 4)



begin training using onecycle policy with max lr of 5e-05...
Epoch 1/4
3334/3334 [==============================] - 1219s 366ms/step - loss: 0.6141 - accuracy: 0.6504 - val_loss: 0.5149 - val_accuracy: 0.7402
Epoch 2/4
3334/3334 [==============================] - 1217s 365ms/step - loss: 0.5095 - accuracy: 0.7409 - val_loss: 0.5133 - val_accuracy: 0.7342
Epoch 3/4
3334/3334 [==============================] - 1218s 365ms/step - loss: 0.3846 - accuracy: 0.8219 - val_loss: 0.4955 - val_accuracy: 0.7738
Epoch 4/4
3334/3334 [==============================] - 1218s 365ms/step - loss: 0.1987 - accuracy: 0.9154 - val_loss: 0.6012 - val_accuracy: 0.7828


Predictions

In [28]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [29]:
db_preds = []
i = 0
while i <len(testsentences):
  x1 = predictor.predict_proba(testsentences[i])
  x2 = predictor.predict_proba(testsentences[i+1])
  if x1[1]>x2[1] and x2[0]>x1[0]:
    db_preds.append(1)
  else:
    db_preds.append(0)
  i += 2

Getting labels with majority voting

In [30]:
output = []
for i in range(len(testsentenceid)):
  if db_preds[i] == bert_pred[i] == gpt_pred[i]:
    output.append([testsentenceid[i],db_preds[i]])

  elif db_preds[i] == bert_pred[i] and bert_pred[i] != gpt_pred[i]:
    output.append([testsentenceid[i],db_preds[i]])
  
  elif db_preds[i] == gpt_pred[i] and db_preds[i] != bert_pred[i]:
    output.append([testsentenceid[i],db_preds[i]])

  elif gpt_pred[i] == bert_pred[i] and db_preds[i] != gpt_pred[i]:
    output.append([testsentenceid[i],bert_pred[i]])

###Accuracy for Bert and Distil Bert and all three combined

In [32]:
df = pd.read_csv (r'SemEval2020-Task4-Commonsense-Validation-and-Explanation/''ALL data''/''Test Data''/subtaskA_gold_answers.csv', header = None)
testl =  df.to_numpy()

In [33]:
db_c = 0
for i in range(1000):
  if db_preds[i] == testl[i][1]:
    db_c+=1
print('DistilBert Accuracy:',db_c/1000*100,'%')

DistilBert Accuracy: 84.89999999999999 %


In [34]:
bc = 0
for i in range(1000):
  if bert_pred[i] == testl[i][1]:
    bc+=1
print('Bert Accuracy:',bc/1000*100,'%')

Bert Accuracy: 83.2 %


In [35]:
c = 0
for i in range(1000):
  if output[i][1] == testl[i][1]:
    c+=1

print('Overall Accuracy:',c/1000*100,'%')

Overall Accuracy: 87.2 %


Generate Output file

In [36]:
df = pd.DataFrame(output)
df.to_csv('sample_data/subtaskA_answers.csv', index=False, header=False)

In [37]:
from google.colab import files
files.download('sample_data/subtaskA_answers.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>